# Project 3: Web APIs & Classification



In [47]:
import pandas as pd
import regex as re
import nltk

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import  word_tokenize
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression

pd.set_option('max_colwidth', 100000)

In [48]:
nomil = pd.read_csv('../csv/nomil_clean.csv')
family = pd.read_csv('../csv/family_clean.csv')

In [49]:
result = pd.merge(nomil, family,how='outer')

In [50]:
result.count()

selftext     1895
subreddit    1895
dtype: int64

## Data Cleaning

In [51]:
def clean_text(text):
    # remove HTML tags and URLs
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*','',text)
    
    # keep only text without punctuation
    text = re.sub(r'[^\w\s]','',text)
    
    # convert text to lowercase
    text = text.strip().lower()

    # split text into a list of words
    token_text = re.split('\W+',text) #W+ --> word chars and dashes permitted
    
    return token_text

In [52]:
#result = clean_text(result['selftext'])
#result.selftext.head(1)

result['token'] = result['selftext'].apply(lambda x: clean_text(x))
result.head(1)

,selftext,subreddit,token
0,"Hello! You may have noticed more flairs around the sub lately. That's because we have listened to user feedback requesting ways to allow more open dialogue between OPs and commenters. To facilitate this, we have implemented a flair system that allows the OP to let our community know their situation and what level of feedback they are comfortable with. For our community, the OP can include a trigger warning with any of these. From now on, we will require all posts to have a flair. Any post that is not will either be taken down temporarily or have a flair assigned by a moderator. If your post is taken down, you will have to choose a flair and then contact the moderators via [ModMail](https://reddit.com/message/compose?to=/r/JUSTNOMIL) to have your post restored. To help you understand the flairs, I've included the descriptions of the flairs that you can find in the [JustNoMIL Wiki.](https://www.reddit.com/r/JUSTNOMIL/wiki/index)\n\nThe new language you will find in the Wiki is:\n\n&gt;An OP can select one of the following flairs to tag a post. You are allowed to tailor your comments to fit *within reason.* Rule 5 remains in effect. Posts that are left without flair are subject to being temporarily removed, or having flair chosen for them, at moderator discretion. There are also trigger warning versions of each flair. **Trigger Warning** flairs dictate that the trigger warning be briefly described in the **first** sentence of the post. \n&gt; \n&gt;Old Story- NO Advice Wanted (*Rather self explanatory*) \n&gt; \n&gt;It's Handled- NO Advice Wanted (*While not exactly historical, the OP has the situation in hand and doesn't need advice.*) \n&gt; \n&gt;RANT- NO Advice Wanted (*Sometimes, you just need to get it out. What you don't need are pointers.*) \n&gt; \n&gt;RANT- Advice Wanted (*Sometimes, you need to get it out and then ask if you've missed something.*) \n&gt; \n&gt;TLC Needed (*For when you've had that hard day and you just need someone to give a damn.*) \n&gt; \n&gt;Advice Please! (*Pointers are welcome.*) \n&gt; \n&gt;LIVE Advice Please! (*For developing situations that need advice in a hurry.*) \n&gt; \n&gt;Gentle Advice Needed (*For those times that you need advice but are feeling a little sensitive.*) \n&gt; \n&gt;Am I the JustNO? (*Worried that you're the one who is behaving badly? Let your fellow community members help you figure that out. MILpologizing allowed within reason.*) \n&gt; \n&gt;Am I Overreacting? (*Is she a JustNo or isn't she? MILpologizing allowed within reason.*) \n&gt; \n&gt;Give It To Me Straight (*For when you need advice with a kick. MILpologizing allowed within reason.*) \n&gt; \n&gt;New User (*Be Gentle.*) \n&gt; \n&gt;Trigger Warnings should be used for Domestic/Interpersonal Violence, Child Abuse of any kind, Sexual Violence of any kind, Graphic descriptions of violent situations, Suicide, Hate Speech and the actions that go along with it, Pregnancy Loss and Loss of a Child.",JUSTNOMIL,"[hello, you, may, have, noticed, more, flairs, around, the, sub, lately, thats, because, we, have, listened, to, user, feedback, requesting, ways, to, allow, more, open, dialogue, between, ops, and, commenters, to, facilitate, this, we, have, implemented, a, flair, system, that, allows, the, op, to, let, our, community, know, their, situation, and, what, level, of, feedback, they, are, comfortable, with, for, our, community, the, op, can, include, a, trigger, warning, with, any, of, these, from, now, on, we, will, require, all, posts, to, have, a, flair, any, post, that, is, not, will, either, be, taken, down, temporarily, or, have, a, flair, ...]"


In [53]:
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [54]:
result['token_stop'] = result['token'].apply(lambda x: [item for item in x if item not in stop])
result.head(1)

,selftext,subreddit,token,token_stop
0,"Hello! You may have noticed more flairs around the sub lately. That's because we have listened to user feedback requesting ways to allow more open dialogue between OPs and commenters. To facilitate this, we have implemented a flair system that allows the OP to let our community know their situation and what level of feedback they are comfortable with. For our community, the OP can include a trigger warning with any of these. From now on, we will require all posts to have a flair. Any post that is not will either be taken down temporarily or have a flair assigned by a moderator. If your post is taken down, you will have to choose a flair and then contact the moderators via [ModMail](https://reddit.com/message/compose?to=/r/JUSTNOMIL) to have your post restored. To help you understand the flairs, I've included the descriptions of the flairs that you can find in the [JustNoMIL Wiki.](https://www.reddit.com/r/JUSTNOMIL/wiki/index)\n\nThe new language you will find in the Wiki is:\n\n&gt;An OP can select one of the following flairs to tag a post. You are allowed to tailor your comments to fit *within reason.* Rule 5 remains in effect. Posts that are left without flair are subject to being temporarily removed, or having flair chosen for them, at moderator discretion. There are also trigger warning versions of each flair. **Trigger Warning** flairs dictate that the trigger warning be briefly described in the **first** sentence of the post. \n&gt; \n&gt;Old Story- NO Advice Wanted (*Rather self explanatory*) \n&gt; \n&gt;It's Handled- NO Advice Wanted (*While not exactly historical, the OP has the situation in hand and doesn't need advice.*) \n&gt; \n&gt;RANT- NO Advice Wanted (*Sometimes, you just need to get it out. What you don't need are pointers.*) \n&gt; \n&gt;RANT- Advice Wanted (*Sometimes, you need to get it out and then ask if you've missed something.*) \n&gt; \n&gt;TLC Needed (*For when you've had that hard day and you just need someone to give a damn.*) \n&gt; \n&gt;Advice Please! (*Pointers are welcome.*) \n&gt; \n&gt;LIVE Advice Please! (*For developing situations that need advice in a hurry.*) \n&gt; \n&gt;Gentle Advice Needed (*For those times that you need advice but are feeling a little sensitive.*) \n&gt; \n&gt;Am I the JustNO? (*Worried that you're the one who is behaving badly? Let your fellow community members help you figure that out. MILpologizing allowed within reason.*) \n&gt; \n&gt;Am I Overreacting? (*Is she a JustNo or isn't she? MILpologizing allowed within reason.*) \n&gt; \n&gt;Give It To Me Straight (*For when you need advice with a kick. MILpologizing allowed within reason.*) \n&gt; \n&gt;New User (*Be Gentle.*) \n&gt; \n&gt;Trigger Warnings should be used for Domestic/Interpersonal Violence, Child Abuse of any kind, Sexual Violence of any kind, Graphic descriptions of violent situations, Suicide, Hate Speech and the actions that go along with it, Pregnancy Loss and Loss of a Child.",JUSTNOMIL,"[hello, you, may, have, noticed, more, flairs, around, the, sub, lately, thats, because, we, have, listened, to, user, feedback, requesting, ways, to, allow, more, open, dialogue, between, ops, and, commenters, to, facilitate, this, we, have, implemented, a, flair, system, that, allows, the, op, to, let, our, community, know, their, situation, and, what, level, of, feedback, they, are, comfortable, with, for, our, community, the, op, can, include, a, trigger, warning, with, any, of, these, from, now, on, we, will, require, all, posts, to, have, a, flair, any, post, that, is, not, will, either, be, taken, down, temporarily, or, have, a, flair, ...]","[hello, may, noticed, flairs, around, sub, lately, thats, listened, user, feedback, requesting, ways, allow, open, dialogue, ops, commenters, facilitate, implemented, flair, system, allows, op, let, community, know, situation, level, feedback, comfortable, community, op, include, trigger, warning, require, posts, flair, post, 

In [55]:
result.columns
feature = ['token_stop', 'subreddit']
result = result[feature]
result.head(3)

,token_stop,subreddit
0,"[hello, may, noticed, flairs, around, sub, lately, thats, listened, user, feedback, requesting, ways, allow, open, dialogue, ops, commenters, facilitate, implemented, flair, system, allows, op, let, community, know, situation, level, feedback, comfortable, community, op, include, trigger, warning, require, posts, flair, post, either, taken, temporarily, flair, assigned, moderator, post, taken, choose, flair, contact, moderators, via, modmailhttpsredditcommessagecomposetorjustnomil, post, restored, help, understand, flairs, ive, included, descriptions, flairs, find, justnomil, wikihttpswwwredditcomrjustnomilwikiindex, new, language, find, wiki, gtan, op, select, one, following, flairs, tag, post, allowed, tailor, comments, fit, within, reason, rule, 5, remains, effect, posts, left, without, flair, subject, temporarily, removed, flair, chosen, moderator, discretion, also, ...]",JUSTNOMIL
1,"[mil, suck, dont, feel, like, making, entire, post, bec, want, vent, crumbs, carpet, moment, post]",JUSTNOMIL
2,"[story, growing, spine, older, one, ill, share, anyone, shining, spine, motivational, missive, maybe, hope, enjoy, ampx200b, ampx200b, baby, like, doesnt, feel, like, terribly, long, ago, kid, ten, things, definitely, changed, since, back, days, didnt, fancy, video, baby, monitors, breathing, sensors, amazing, fancy, things, new, moms, get, days, simple, baby, monitor, base, babys, room, receiver, either, bedroom, living, room, wherever, pretty, good, monitor, could, definitely, hear, going, babys, room, story, begins, ampx200b, parents, visiting, didnt, know, justno, time, knew, biological, father, abusive, hence, restraining, orders, mom, much, better, didnt, ever, think, question, said, became, mother, bets, lost, first, baby, nobody, going, ...]",JUSTNOMIL


## Lemmatization & Stop word

In [56]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    lemma_words = [lemmatizer.lemmatize(word) for word in text]
    return lemma_words

In [57]:
result['token_stop1'] = result['token_stop'].apply(lambda x: lemmatize_words(x))

In [58]:
result.head(1)

,token_stop,subreddit,token_stop1
0,"[hello, may, noticed, flairs, around, sub, lately, thats, listened, user, feedback, requesting, ways, allow, open, dialogue, ops, commenters, facilitate, implemented, flair, system, allows, op, let, community, know, situation, level, feedback, comfortable, community, op, include, trigger, warning, require, posts, flair, post, either, taken, temporarily, flair, assigned, moderator, post, taken, choose, flair, contact, moderators, via, modmailhttpsredditcommessagecomposetorjustnomil, post, restored, help, understand, flairs, ive, included, descriptions, flairs, find, justnomil, wikihttpswwwredditcomrjustnomilwikiindex, new, language, find, wiki, gtan, op, select, one, following, flairs, tag, post, allowed, tailor, comments, fit, within, reason, rule, 5, remains, effect, posts, left, without, flair, subject, temporarily, removed, flair, chosen, moderator, discretion, also, ...]",JUSTNOMIL,"[hello, may, noticed, flair, around, sub, lately, thats, listened, user, feedback, requesting, way, allow, open, dialogue, ops, commenters, facilitate, implemented, flair, system, allows, op, let, community, know, situation, level, feedback, comfortable, community, op, include, trigger, warning, require, post, flair, post, either, taken, temporarily, flair, assigned, moderator, post, taken, choose, flair, contact, moderator, via, modmailhttpsredditcommessagecomposetorjustnomil, post, restored, help, understand, flair, ive, included, description, flair, find, justnomil, wikihttpswwwredditcomrjustnomilwikiindex, new, language, find, wiki, gtan, op, select, one, following, flair, tag, post, allowed, tailor, comment, fit, within, reason, rule, 5, remains, effect, post, left, without, flair, subject, temporarily, removed, flair, chosen, moderator, discretion, also, ...]"


In [59]:
result['token_stop1'] = result['token_stop1'].apply(lambda x: ",".join(x))

In [60]:
result.head(1)

,token_stop,subreddit,token_stop1
0,"[hello, may, noticed, flairs, around, sub, lately, thats, listened, user, feedback, requesting, ways, allow, open, dialogue, ops, commenters, facilitate, implemented, flair, system, allows, op, let, community, know, situation, level, feedback, comfortable, community, op, include, trigger, warning, require, posts, flair, post, either, taken, temporarily, flair, assigned, moderator, post, taken, choose, flair, contact, moderators, via, modmailhttpsredditcommessagecomposetorjustnomil, post, restored, help, understand, flairs, ive, included, descriptions, flairs, find, justnomil, wikihttpswwwredditcomrjustnomilwikiindex, new, language, find, wiki, gtan, op, select, one, following, flairs, tag, post, allowed, tailor, comments, fit, within, reason, rule, 5, remains, effect, posts, left, without, flair, subject, temporarily, removed, flair, chosen, moderator, discretion, also, ...]",JUSTNOMIL,"hello,may,noticed,flair,around,sub,lately,thats,listened,user,feedback,requesting,way,allow,open,dialogue,ops,commenters,facilitate,implemented,flair,system,allows,op,let,community,know,situation,level,feedback,comfortable,community,op,include,trigger,warning,require,post,flair,post,either,taken,temporarily,flair,assigned,moderator,post,taken,choose,flair,contact,moderator,via,modmailhttpsredditcommessagecomposetorjustnomil,post,restored,help,understand,flair,ive,included,description,flair,find,justnomil,wikihttpswwwredditcomrjustnomilwikiindex,new,language,find,wiki,gtan,op,select,one,following,flair,tag,post,allowed,tailor,comment,fit,within,reason,rule,5,remains,effect,post,left,without,flair,subject,temporarily,removed,flair,chosen,moderator,discretion,also,trigger,warning,version,flair,trigger,warning,flair,dictate,trigger,warning,briefly,described,first,sentence,post,gt,gtold,story,advice,wanted,rather,self,explanatory,gt,gtits,handled,advice,wanted,exactly,historical,op,situation,hand,doesnt,need,advice,gt,gtrant,advice,wanted,sometimes,need,get,dont,need,pointer,gt,gtrant,advice,wanted,sometimes,need,get,ask,youve,missed,something,gt,gttlc,needed,youve,hard,day,need,someone,give,damn,gt,gtadvice,please,pointer,welcome,gt,gtlive,advice,please,developing,situation,need,advice,hurry,gt,gtgentle,advice,needed,time,need,advice,feeling,little,sensitive,gt,gtam,justno,worried,youre,one,behaving,badly,let,fellow,community,member,help,figure,milpologizing,allowed,within,reason,gt,gtam,overreacting,justno,isnt,milpologizing,allowed,within,reason,gt,gtgive,straight,need,advice,kick,milpologizing,allowed,within,reason,gt,gtnew,user,gentle,gt,gttrigger,warning,used,domesticinterpersonal,violence,child,abuse,kind,sexual,violence,kind,graphic,description,violent,situation,suicide,hate,speech,action,go,along,pregnancy,loss,loss,child"


In [61]:
feature = ['token_stop1', 'subreddit']
result = result[feature]
result.head(1)

,token_stop1,subreddit
0,"hello,may,noticed,flair,around,sub,lately,thats,listened,user,feedback,requesting,way,allow,open,dialogue,ops,commenters,facilitate,implemented,flair,system,allows,op,let,community,know,situation,level,feedback,comfortable,community,op,include,trigger,warning,require,post,flair,post,either,taken,temporarily,flair,assigned,moderator,post,taken,choose,flair,contact,moderator,via,modmailhttpsredditcommessagecomposetorjustnomil,post,restored,help,understand,flair,ive,included,description,flair,find,justnomil,wikihttpswwwredditcomrjustnomilwikiindex,new,language,find,wiki,gtan,op,select,one,following,flair,tag,post,allowed,tailor,comment,fit,within,reason,rule,5,remains,effect,post,left,without,flair,subject,temporarily,removed,flair,chosen,moderator,discretion,also,trigger,warning,version,flair,trigger,warning,flair,dictate,trigger,warning,briefly,described,first,sentence,post,gt,gtold,story,advice,wanted,rather,self,explanatory,gt,gtits,handled,advice,wanted,exactly,historical,op,situation,hand,doesnt,need,advice,gt,gtrant,advice,wanted,sometimes,need,get,dont,need,pointer,gt,gtrant,advice,wanted,sometimes,need,get,ask,youve,missed,something,gt,gttlc,needed,youve,hard,day,need,someone,give,damn,gt,gtadvice,please,pointer,welcome,gt,gtlive,advice,please,developing,situation,need,advice,hurry,gt,gtgentle,advice,needed,time,need,advice,feeling,little,sensitive,gt,gtam,justno,worried,youre,one,behaving,badly,let,fellow,community,member,help,figure,milpologizing,allowed,within,reason,gt,gtam,overreacting,justno,isnt,milpologizing,allowed,within,reason,gt,gtgive,straight,need,advice,kick,milpologizing,allowed,within,reason,gt,gtnew,user,gentle,gt,gttrigger,warning,used,domesticinterpersonal,violence,child,abuse,kind,sexual,violence,kind,graphic,description,violent,situation,suicide,hate,speech,action,go,along,pregnancy,loss,loss,child",JUSTNOMIL


In [62]:
#Baseline
result['subreddit'].value_counts(normalize=True)

JUSTNOMIL    0.501319
family       0.498681
Name: subreddit, dtype: float64

## Train/Test split

In [63]:
#Assignment of target
X = result['token_stop1']
y = result['subreddit']

#Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

### 1) Logistic Regression

In [64]:
## Baseline Logistic Regression
#Creation of pipeline
logpipe = Pipeline([('cv', CountVectorizer()),
                 ('tfidf',TfidfTransformer()),
                 ('lr', LogisticRegression())])
# Fit your model
logpipe.fit(X_train, y_train)

# Training score
print(logpipe.score(X_train, y_train))
# Test score
print(logpipe.score(X_test, y_test))

y_pred_ = logpipe.predict(X_test)
print(classification_report(y_test,y_pred_))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9627023223082336
0.8881856540084389
              precision    recall  f1-score   support

   JUSTNOMIL       0.91      0.86      0.89       238
      family       0.86      0.92      0.89       236

   micro avg       0.89      0.89      0.89       474
   macro avg       0.89      0.89      0.89       474
weighted avg       0.89      0.89      0.89       474



### Parameters tuning: Logistic Regression

In [65]:
%%time
pipe_params = {
    'cv__max_features': [50, 100, 200, 300],
    'cv__min_df': [2, 3],
    'cv__max_df': [.9, .95],
    'cv__ngram_range': [(1,1), (1,2), (1,3), (1,4)],
    #'cv__penalty': ['l1', 'l2']
    }
                        
gs = GridSearchCV(logpipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)
y_pred_log = gs.predict(X_test)
print(classification_report(y_test,y_pred_log))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8676988036593948
{'cv__max_df': 0.9, 'cv__max_features': 300, 'cv__min_df': 2, 'cv__ngram_range': (1, 1)}
              precision    recall  f1-score   support

   JUSTNOMIL       0.89      0.84      0.87       238
      family       0.85      0.90      0.87       236

   micro avg       0.87      0.87      0.87       474
   macro avg       0.87      0.87      0.87       474
weighted avg       0.87      0.87      0.87       474

CPU times: user 4min 47s, sys: 5.93 s, total: 4min 53s
Wall time: 4min 53s


### 2) MultiNomial NB

In [66]:
# MultinomialNB model

pipenb = Pipeline([('cv', CountVectorizer()),
                 #('tfidf',TfidfTransformer()), # Affect test scores by 10%
                 ('mnb', MultinomialNB())])

pipenb.fit(X_train, y_train)
print('train score:', pipenb.score(X_train, y_train))
print('test score:', pipenb.score(X_test, y_test))

y_pred__ = pipenb.predict(X_test)
print(classification_report(y_test,y_pred__))

train score: 0.9732582688247713
test score: 0.8227848101265823
              precision    recall  f1-score   support

   JUSTNOMIL       0.77      0.93      0.84       238
      family       0.91      0.71      0.80       236

   micro avg       0.82      0.82      0.82       474
   macro avg       0.84      0.82      0.82       474
weighted avg       0.84      0.82      0.82       474



### Parameters tuning: MultiNomial NB

In [67]:
%%time

params = { 
#     'cvec__ngram_range': [(1, 1), (1, 2)]
    'cv__max_features': [50, 100, 200, 300], 
    'cv__ngram_range': [(1, 1),(1,2), (1,3), (1,4)], 
    'mnb__alpha': [0.25, 0.5, 0.75, 1]
    # min_df
    # max_df
}
gs = GridSearchCV(pipenb, param_grid=params, cv=3, verbose=1)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)
y_pred_nb = gs.predict(X_test)
print(classification_report(y_test,y_pred_nb))

Fitting 3 folds for each of 64 candidates, totalling 192 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 192 out of 192 | elapsed:  5.0min finished


0.8353272343420126
{'cv__max_features': 300, 'cv__ngram_range': (1, 4), 'mnb__alpha': 0.25}
              precision    recall  f1-score   support

   JUSTNOMIL       0.84      0.83      0.83       238
      family       0.83      0.83      0.83       236

   micro avg       0.83      0.83      0.83       474
   macro avg       0.83      0.83      0.83       474
weighted avg       0.83      0.83      0.83       474

CPU times: user 5min, sys: 6.56 s, total: 5min 7s
Wall time: 5min


## Conclusion

Logistic regression out perform Multinomial NB model by 3%